In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l



In [2]:
#为了仅将有意义的词元作为值来获取注意力汇聚， 
#可以指定一个有效序列长度（即词元的个数）， 
#以便在计算softmax时过滤掉超出指定范围的位置。 
#下面的masked_softmax函数 实现了这样的掩蔽softmax操作（masked softmax operation），'
#其中任何超出有效长度的位置都被掩蔽并置为0。
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    #X 3d张量， valid lens 1d或2d张量
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1: # 1d
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else: # 2d
            valid_lens = valid_lens.reshape(-1)
        # 最后一个轴上被掩蔽的元素使用一个非常大的负值来替换，这样过softmax函数后输出为0
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens, value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)
        


In [3]:
masked_softmax(torch.rand(2,2,4), torch.tensor([2,3]))

tensor([[[0.4587, 0.5413, 0.0000, 0.0000],
         [0.6785, 0.3215, 0.0000, 0.0000]],

        [[0.3623, 0.3248, 0.3129, 0.0000],
         [0.4358, 0.2873, 0.2769, 0.0000]]])

In [4]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.2296, 0.4126, 0.3578, 0.0000]],

        [[0.4201, 0.5799, 0.0000, 0.0000],
         [0.2217, 0.2035, 0.2645, 0.3103]]])